In [1]:
import pandas as pd
import numpy as np
from progress.bar import Bar
from utils.load import load_data_methodology_cutting
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
from utils.metrics import f1, confusion_matrix_plot
from utils.tuner import CVTuner
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
import joblib
import keras_tuner
import tensorflow as tf

2024-01-06 20:45:14.279008: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-06 20:45:14.922815: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-06 20:45:14.923268: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-06 20:45:14.925917: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-06 20:45:15.192939: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-06 20:45:15.209189: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
df = pd.read_csv("../Metadata/data_curated.csv")
df.set_index("File", inplace=True)
df.head()

,Class
File,
S1V3C1M3A Clip166.npy,1
S1V1C1M1A Clip18.npy,1
S1V4C1M4A Clip241.npy,1
S1V4C1M4A Clip239.npy,1
S1V4C1M4A Clip245.npy,1


In [3]:
X_train, y_train, X_val, y_val = load_data_methodology_cutting(df,30,"M1")

In [4]:
X_train.shape

(2250, 30, 1662)

In [5]:
y_train.shape

(2250, 6)

In [6]:
X_val.shape

(750, 30, 1662)

In [7]:
y_val.shape

(750, 6)

# Cross Validation

In [8]:
cv = MultilabelStratifiedKFold(n_splits=5)
joblib.dump(cv, './cv/cv.joblib')

['./cv/cv.joblib']

# Hyper Parameter Tunning Recurrent Neural Networks

In [9]:
def build_model_LSTM(hp):
    
    model = Sequential()
    model.add(LSTM(hp.Int('input_units',min_value=30,max_value=200,step=20), return_sequences=True, activation='relu', input_shape=(30,1662)))
    model.add(LSTM(hp.Int('LSTM_1',min_value=100,max_value=200,step=20), return_sequences=True, activation='relu'))
    model.add(LSTM(hp.Int('LSTM_2',min_value=30,max_value=100,step=20), return_sequences=False, activation='relu'))
    model.add(Dense(hp.Int(f"Dense_units_1",min_value=10,max_value=60,step=10), activation='relu'))
    model.add(Dense(hp.Int(f"Dense_units_2",min_value=10,max_value=30,step=10), activation='relu'))
    model.add(Dense(6, activation='softmax'))

    opt = tf.keras.optimizers.Nadam(clipnorm = 1.0)
    model.compile(optimizer=opt, loss="binary_crossentropy",metrics=[f1])
    return model

In [12]:
tuner_LSTM = CVTuner(
                    data_cv= cv,
                    goal = 'f1',
                    hypermodel=build_model_LSTM,
                    oracle=keras_tuner.oracles.BayesianOptimizationOracle(
                    objective=keras_tuner.Objective('f1',direction = "max"),
                        max_trials=5                  
                    ),
                    directory='./experiments/',
                    project_name = 'LSTM',
                    overwrite=True
                    )

In [13]:
tuner_LSTM.search(X_train, y_train, 128, epochs=40)

Trial 5 Complete [00h 11m 28s]
f1: 0.28008725345134733

Best f1 So Far: 0.3620250649750233
Total elapsed time: 01h 16m 09s
INFO:tensorflow:Oracle triggered exit


In [15]:
df_LSTM = pd.DataFrame(tuner_LSTM.trial_scores)

In [16]:
df_LSTM.to_csv("./experiments/LSTM.csv")

In [ ]:
#disp = confusion_matrix_plot(y_val, y_pred)
#disp.plot()